# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [5]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [6]:
data = pd.read_csv('/datasets/insurance.csv')
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [7]:
display(data.info())
display(data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Пропусков нет, данные не требуют доработки.

In [8]:
X = data.drop(columns='Страховые выплаты')
y = data['Страховые выплаты']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=12345)
X.shape

(5000, 4)

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** b. Не изменится.

**Обоснование:** Нужно доказать, что $a1 = XPw = a$

Новая матрица признаков $X$ будет равна $XP$. Подставим её в формулу обучения.

$$
w1 = ((XP)^T XP)^{-1} (XP)^T y
$$

Раскроем скобки:

$$
w1 = (P^TX^TXP)^{-1}P^TX^Ty
$$

Далее раскроем скобки обратной матрицы:

$$
w1 = P^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^Ty
$$

Здесь замечаем выражение $(P^T)^{-1}P^T$. Его можно сократить, так как $P$ здесь умножается на $P^{-1}$, то есть на обратную матрицу. В результате такого произведения получается единичная матрица, а при умножении матрицы на единичную матрицу получается та же самая матрица.

Получаем уже знакомую формулу $w$:

$$
w1 = P^{-1}(X^TX)^{-1}X^Ty
$$ 

$$
w1 = P^{-1}w
$$

Чтобы доказать неизменность предсказаний, подставим новую формулу обучения в формулу предсказаний:

$$
a1 = XPw1 = XPP^{-1}w
$$

Снова видим умножение матрицы на обратную, поэтому сокращаем $PP^{-1}$:

$$
a1 = Xw = a
$$

Что и требовалось доказать.

## Алгоритм преобразования

_Предложите алгоритм преобразования данных для решения задачи. Обоснуйте, почему качество линейной регрессии не поменяется._

**Алгоритм**

1. Сгенерировать случайную матрицу при помощи функции randint(). У случайной матрицы должно быть такое же количество строк, сколько столбцов у матрицы признаков (у нас 4 признака). К тому же она должна быть квадратной, так как обратимыми могут быть только квадратные матрицы.
2. Проверить матрицу на обратимость.
3. Умножить случайную матрицу на матрицу признаков.
4. Обучить линейную регрессию.
5. Посчитать R2_score на зашифрованных и незашифрованных признаках и сравнить метрики.

**Обоснование**
Метрика R2_score сравнивает таргет и предсказания. Так как выше было доказано, что предсказания не изменяются при умножении на обратимую матрицу, этим можно обосновать и неизменность метрики. 

## Проверка алгоритма

In [9]:
#создаем квадратную матрицу с размером, равных числу признаков:
P = np.random.randint(100, size = (4, 4))
P

array([[21, 36, 76,  5],
       [89, 72,  0, 72],
       [44, 52, 90, 47],
       [34,  3, 24, 89]])

In [10]:
#проверяем на обратимость:
np.linalg.inv(P)

array([[ 0.21884922,  0.03417392, -0.20270004,  0.06710262],
       [-0.18106339, -0.01611283,  0.17075591, -0.06696727],
       [ 0.0443387 , -0.00100507, -0.03012541,  0.01423108],
       [-0.08945856, -0.01224105,  0.07980386, -0.01597902]])

In [11]:
model_original = LinearRegression()
model_original.fit(X_train, y_train)
predicted_original= model_original.predict(X_test)
r2_original = r2_score(y_test, predicted_original)
print('R2 на исходных данных:', round(r2_original, 4))

R2 на исходных данных: 0.4352


In [12]:
#умножим признаки из тренировочной и тестовой выборки на созданную матрицу и создадим зашифрованные признаки:
X_train_new = X_train@P
X_test_new = X_test@P
display(X_train_new.shape)
display(X_test_new.shape)

(3750, 4)

(1250, 4)

In [13]:
model_new = LinearRegression()
model_new.fit(X_train_new, y_train)
predicted_new = model_new.predict(X_test_new)
r2_new = r2_score(y_test, predicted_new)
print('R2 на преобразованных данных:', round(r2_new, 4))

R2 на преобразованных данных: 0.4352


In [14]:
print(r2_original)
print(r2_new)

0.43522757127026535
0.4352275712699898


Метрика совсем не значительно изменилась с преобразованием данных, значит, нам удалось зашифровать данные пользователей и не ухудшить тем самым качество работы модели.

## Вывод

1. Загрузили и исследовали данные, предобработка не понадобилась.
2. Теоретически доказали неизменность предсказаний при умножении признаков на обратимую матрицу.
3. Предложили алгоритм преобразования данных.
4. Проверили его, умножив признаки в тестовой и тренировочной выборках на квадратную обратимую матрицу P.
5. Сравнили метрики R2_score моделей, обученных на преобразованных и исходных данных и получили почти идентичные результаты.